# A Python Implementation of FNGBS

Original Paper: [A Fast Neighborhood Grouping Method for Hyperspectral Band Selection](https://ieeexplore.ieee.org/document/9153939)
Authors: Qi Wang ,Qiang Li ,Xuelong Li

This implementation didn't **pefectly** replicate the result of original paper in Indian Pines dataset and I have no idea why so far. So if you have any idea please summit an issue (or even an PR), thanks!

Most of the variables name are referred from the params in the fomulas in the paper.

In [ ]:
import cv2
import spectral
import numpy as np
import scipy.signal as sig
from scipy.spatial import KDTree

#Enter the references map path here
ref = cv2.imread('data/10_4231_R7RX991C/documentation/Site3_Project_and_Ground_Reference_Files/19920612_AVIRIS_IndianPine_Site3_gr.tif')
ref = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)

disableBands = np.array([104,105,106,107,108,150,151,152,153,154,155,156,157,158,159,160,161,162,163,220])-1

#Enter the 92AV3C.lan path here
himg = spectral.open_image('data/10_4231_R7RX991C/92AV3C.lan')
harr = himg.load()
activeBands = list(range(harr.shape[2]))
for db in disableBands:
    activeBands.remove(db)
harr = harr[:,:,activeBands]
harr /= harr.max()
harr = np.array(harr*255, np.uint8)
locs = np.argwhere(ref!=255) #N #pixels
bands = harr.shape[2] #L #bands
tgtBands = 8 #M Target Bands


In [ ]:
hvec = np.zeros([bands, len(locs)], np.uint8)
for b in range(bands):
    for idx,loc in enumerate(locs):
        hvec[b,idx] = harr[loc[0], loc[1], b]

#hvec = hvec[:,:9323]

In [ ]:
def getG(m, M, L):
    if m == 0:
        return 0
    elif m == M:
        return L
    else:
        return int((m)/M*(L - L%M))


def getNeighbor(p,m,M,L):
    if m==0:
        return (0, p[m+1])
    elif m < M-1:
        return (p[m-1]+1, p[m+1])
    else:
        return (p[m-1], L)

def getDist(a,b):
    return np.linalg.norm(a-b)

class X_m():
    def __init__(self, M:int, L:int, g=[], D=None, hvec=None):
        self.M = M
        self.L = L
        self.hvec = hvec
        if not len(g):
            g = []
            for m in range(0,M+1):
                g.append(getG(m,M,L))
        
        X = []
        for m in range(M):
            X.append([])
            for i in range(g[m],g[m+1]):
                X[m].append(i)

        self.g = g
        self.group = X

        p = []

        for m in range(M):
            p.append(int((m+1)*L/M - L/2/M))

        self.p = p

        neighbor = []

        for m in range(M):
            neighbor.append(getNeighbor(p,m,M,L))

        self.neighbor = neighbor

        if not isinstance(D,np.ndarray):
            D = np.zeros([self.L, self.L])

            for a in range(L):
                print(f'\rComputing Distance {a}/{L}',end='')
                for b in range(a,L):
                    if a-b:
                        dist = getDist(hvec[a], hvec[b])/hvec.shape[1]
                        D[a,b] = dist
                        D[b,a] = dist
            print()
        self.D = D

def FPA(groups:X_m, inter:int = 10):
    R = np.zeros([groups.L])
    R.fill(np.inf)
    T = np.zeros([groups.L], np.uint8)
    D = groups.D
    i = 0
    lastT = np.array(T)
    while(i<inter):
        for m in range(groups.M):
            p_m = groups.p[m]
            for j in range(*groups.neighbor[m]):
                if D[p_m, j] < R[j]:
                    R[j] = D[p_m, j]
                    T[j] = m

        T = np.array(sig.medfilt(T),np.uint8)
        diff = np.abs(T) - np.abs(lastT)
        if diff.sum() < 0.5:
            break
        lastT = np.array(T)
        groups = X_m2(groups.M,groups.L,getGfromT(T), groups.D)
        i+=1


    return groups

                
def getGfromT(T):
    nList = np.unique(T)
    g = [0]
    for n in nList:
        g.append(np.where(T==n)[0].max())

    return g


def kNN(groups, x_u, k):
    Dlist = np.array([groups.D[x_u]]).T
    DTree = KDTree(Dlist)
    return DTree.query([0],k=k)[1]

def SNN(groups, x_u, x_v, k):
    # Number of Shared Neighbor Elements
    return len(np.unique(np.concatenate([kNN(groups, x_u, k),kNN(groups, x_v, k)])))

def localDensity(groups, x_u, k):
    rho = 0
    for x_v in kNN(groups, x_u, k):
        rho += np.exp(- groups.D[x_u,x_v] / (SNN(groups,x_u,x_v,k) + 1) )
    
    return rho

def getEntrophy(hvec, Z=0.1):
    idxs = np.random.choice(range(hvec.shape[1]), size=int(hvec.shape[1]*Z))
    H = np.zeros([hvec.shape[0]], np.float)
    for band in range(hvec.shape[0]):
        p = np.zeros([255])
        for idx in idxs:
            p[hvec[band, idx]] += 1

        p /= int(hvec.shape[1]*Z)
        
        for idx,bar in enumerate(p):
            H[band] -= 0 if bar==0 else bar * np.log2(bar)

    return np.array(H, np.float)

def normalize(x:np.ndarray):
    try:
        x_min, x_max = np.min(x), np.max(x)
        x -= x_min
        x /= x_max
    except TypeError:
        x_min, x_max = x.min(), x.max()
        x -= x_min
        x /= x_max
    return x

In [ ]:
groups = X_m(tgtBands,bands,hvec=hvec)
print(groups.g)
print(groups.p)
print(groups.neighbor)

In [ ]:
groups = FPA(groups)
print(groups.g)
print(groups.p)
print(groups.neighbor)

In [ ]:
k = 5
H = getEntrophy(hvec)
ld = np.zeros(groups.L)
for band in range(groups.L):
    ld[band] = localDensity(groups, band, k)

H = normalize(H)
ld = normalize(ld)

In [ ]:
selected = []

for group in groups.group:
    sortedgroup = group[:]
    if len(group) > k:
        sortedgroup.sort(key=lambda x: H[x] * ld[x], reverse=True)
    else:
        sortedgroup.sort(key=lambda x: H[x], reverse=True)
    selected.append(sortedgroup[0])

selected

Partition: [26, 41, 70, 90, 111, 125, 161, 186]
- - KNN: (0.6989332728098048, 0.6140638717858085, 0.6989303423060612, 0.6989332728098048)
- - SVM: (0.6966636404902405, 0.624601571743703, 0.6966607190346674, 0.6966636404902405)